In [1]:
# general tools
import os
import sys
from glob import glob

# data tools
import re
import time
import h5py
import random
import numpy as np
import scipy.ndimage
from random import shuffle
from tensorflow import keras
from datetime import datetime, timedelta

#tf.config.run_functions_eagerly(True)

sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du
import model_utils as mu

2023-04-30 22:06:14.838024: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# import importlib
# importlib.reload(mu)

In [4]:
def verif_metric(VALID_target, Y_pred):
    BS = np.mean((VALID_target.ravel() - Y_pred.ravel())**2)
    metric = BS
    return metric

def neighbour_leads(lead):
    out = [lead-2, lead-1, lead, lead+1]
    flag_shift = [0, 0, 0, 0]
    
    for i in range(4):
        if out[i] < 0:
            out[i] = 24+out[i]
            flag_shift[i] = -1
        if out[i] > 23:
            out[i] = out[i]-24
            flag_shift[i] = +1
            
    return out, flag_shift


def filename_to_loc(filenames):
    lead_out = []
    indx_out = []
    indy_out = []
    day_out = []
    
    for i, name in enumerate(filenames):
        
        nums = re.findall(r'\d+', name)
        
        lead = int(nums[-1])
        indy = int(nums[-2])
        indx = int(nums[-3])
        day = int(nums[-4])
      
        indx_out.append(indx)
        indy_out.append(indy)
        day_out.append(day)
        lead_out.append(lead)
        
    return np.array(indx_out), np.array(indy_out), np.array(day_out), np.array(lead_out)

In [5]:
ind_pick_from_batch = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

with h5py.File(save_dir+'CNN_Validation_basic.hdf', 'r') as h5io:
    VALID_input_64 = h5io['VALID_input_64'][...]
    VALID_target = h5io['VALID_target'][...]

**base basic**

In [9]:
# ==================== #
weights_round = 1
save_round = 2
seeds = 711 #777
model_prefix_load = 'RE3_smooth_basic{}'.format(weights_round) #False
model_prefix_save = 'RE3_smooth_basic{}'.format(save_round)
N_vars = L_vars = 15
lr = 1e-4
# ==================== #

In [10]:
# ----------------------------------------------------------------- #
# model and weights
model_head = mu.create_model_head(input_shape=(128,), N_node=64)
model_base = mu.create_model_base(input_shape=(64, 64, 15), depths=[3, 3, 27, 3], projection_dims=[32, 64, 96, 128], first_pool=4)

IN = keras.layers.Input(shape=(64, 64, 15))

VEC = model_base(IN)
OUT = model_head(VEC)

model_final = keras.models.Model(inputs=IN, outputs=OUT)

In [11]:
# ============================================= #
# Weights

if weights_round > 0:
    if model_prefix_load:
        W_old = mu.dummy_loader('/glade/work/ksha/NCAR/Keras_models/{}/'.format(model_prefix_load))
        model_final.set_weights(W_old)
    
model_final.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), optimizer=keras.optimizers.Adam(lr=lr))


In [12]:
# ----------------------------------------------------------------- #
# model training loop
Y_pred = model_final.predict([VALID_input_64])
record_temp = verif_metric(VALID_target, Y_pred)

2023-04-29 08:36:11.774577: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-29 08:36:11.803079: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2300000000 Hz
2023-04-29 08:36:16.845970: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-29 08:36:17.856226: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [13]:
# Change based on smoothed labels
print(record_temp)

0.025065149177196624


In [6]:
weights_round = 2
save_round = 3
seeds = 711 #777
model_prefix_load = 'RE3_smooth_large{}'.format(weights_round) #False
model_prefix_save = 'RE3_smooth_large{}'.format(save_round)

In [7]:
# ----------------------------------------------------------------- #
# model and weights
model_head = mu.create_model_head(input_shape=(512,), N_node=64)
model_base = mu.create_model_large(input_shape=(64, 64, 15), depths=[3, 3, 27, 3], projection_dims=[96, 128, 256, 512], first_pool=4)

IN = keras.layers.Input(shape=(64, 64, 15))

VEC = model_base(IN)
OUT = model_head(VEC)

model_final = keras.models.Model(inputs=IN, outputs=OUT)

2023-04-29 22:34:10.262798: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-29 22:34:10.275299: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-29 22:34:10.434720: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:b3:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-04-29 22:34:10.434795: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-04-29 22:34:10.627398: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-29 22:34:10.627467: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

In [9]:
# Weights
if weights_round > 0:
    if model_prefix_load:
        W_old = mu.dummy_loader('/glade/work/ksha/NCAR/Keras_models/{}/'.format(model_prefix_load))
        model_final.set_weights(W_old)
    
#model_final.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), optimizer=keras.optimizers.Adam(lr=lr))

In [10]:
# model training loop
Y_pred = model_final.predict([VALID_input_64])
record_temp = verif_metric(VALID_target, Y_pred)

2023-04-29 22:35:18.455457: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-29 22:35:18.488670: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2300000000 Hz
2023-04-29 22:35:22.833349: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-29 22:35:23.391738: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [11]:
# Change based on smoothed labels
print(record_temp)

0.024178190259442508


In [10]:
weights_round = 3
save_round = 4
seeds = 711 #777
model_prefix_load = 'RE3_smooth_vgg{}'.format(weights_round) #False
model_prefix_save = 'RE3_smooth_vgg{}'.format(save_round)

In [11]:
# ----------------------------------------------------------------- #
# model and weights
model_head = mu.create_model_head(input_shape=(512,), N_node=64)
model_base = mu.create_model_vgg(input_shape=(64, 64, 15), channels=[96, 128, 256, 512])

IN = keras.layers.Input(shape=(64, 64, 15))

VEC = model_base(IN)
OUT = model_head(VEC)

model_final = keras.models.Model(inputs=IN, outputs=OUT)


# ============================================= #
# Weights
if weights_round > 0:
    if model_prefix_load:
        W_old = mu.dummy_loader('/glade/work/ksha/NCAR/Keras_models/{}/'.format(model_prefix_load))
        model_final.set_weights(W_old)
    
#model_final.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), optimizer=keras.optimizers.Adam(lr=lr))


In [12]:
# model training loop
Y_pred = model_final.predict([VALID_input_64])
record_temp = verif_metric(VALID_target, Y_pred)

In [13]:
# Change based on smoothed labels
print(record_temp)

0.0230026736801435


In [9]:
# Change based on smoothed labels
print(record_temp)

0.02257461099434288
